In [1]:
#!pip install pyhive[presto]
#!pip install minio
from datetime import datetime
import pandas as pd
import numpy as np
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
# import ibm_boto3
# from ibm_botocore.client import Config, ClientError
# from ibm_watson_studio_lib import access_project_or_space
import os
import io

from minio import Minio
from minio.error import S3Error

# wslib = access_project_or_space()

# params = { '_INSECURE_disable_TLS_certificate_verification' : True }


In [2]:
PRESTO_ADMIN_PWD = os.environ.get("PRESTO_ADMIN_PWD", "none")   
prestoURL = "presto://admin:" + PRESTO_ADMIN_PWD + "@ibm-lh-lakehouse-presto-01-presto-svc.cpd.svc.cluster.local:8443"

engine = create_engine(prestoURL, connect_args={'protocol': 'https'})
conn = engine.connect()

select_stmt = """
SELECT t1.TICKER, t1.ISIN, t1.industry,
COALESCE(t1.TR_ANALYTICCO2, t2.TR_ANALYTICCO2) AS TR_ANALYTICCO2,
COALESCE(t1.TR_ANALYTICTOTALWASTE, t2.TR_ANALYTICTOTALWASTE) AS TR_ANALYTICTOTALWASTE,
COALESCE(t1.TR_ENVPRODUCTS, t2.TR_ENVPRODUCTS) AS TR_ENVPRODUCTS,
COALESCE(t1.TR_ANALYTICENVRND, t2.TR_ANALYTICENVRND) AS TR_ANALYTICENVRND,
COALESCE(t1.TR_ANALYTICWATERUSE, t2.TR_ANALYTICWATERUSE) AS TR_ANALYTICWATERUSE,
COALESCE(t1.TR_ANALYTICENERGYUSE, t2.TR_ANALYTICENERGYUSE) AS TR_ANALYTICENERGYUSE,
COALESCE(t1.TR_POLICYHUMANRIGHTS, t2.TR_POLICYHUMANRIGHTS) AS TR_POLICYHUMANRIGHTS,
COALESCE(t1.TR_POLICYRESPONSIBLEMARKETING, t2.TR_POLICYRESPONSIBLEMARKETING) AS TR_POLICYRESPONSIBLEMARKETING,
COALESCE(t1.TR_PRODUCTQUALITYMONITORING, t2.TR_PRODUCTQUALITYMONITORING) AS TR_PRODUCTQUALITYMONITORING,
COALESCE(t1.TR_POLICYDATAPRIVACY, t2.TR_POLICYDATAPRIVACY) AS TR_POLICYDATAPRIVACY,
COALESCE(t1.TR_WOMENEMPLOYEES, t2.TR_WOMENEMPLOYEES,t3.TR_WOMENEMPLOYEES) AS TR_WOMENEMPLOYEES,
COALESCE(t1.TR_AVGTRAININGHOURS, t2.TR_AVGTRAININGHOURS,t3.TR_AVGTRAININGHOURS) AS TR_AVGTRAININGHOURS,
COALESCE(t1.TR_TRADEUNIONREP, t2.TR_TRADEUNIONREP,t3.TR_TRADEUNIONREP) AS TR_TRADEUNIONREP,
COALESCE(t1.TR_ANALYTICLOSTDAYS, t2.TR_ANALYTICLOSTDAYS,t3.TR_ANALYTICLOSTDAYS) AS TR_ANALYTICLOSTDAYS
FROM db2datamon.admin.esg1 t1 
FULL OUTER JOIN datamon.datamon.esg2 t2 ON t1.ISIN = t2.ISIN 
FULL OUTER JOIN datamon.datamon.workforce t3 ON t1.ISIN = t3.ISIN
"""

%time source1 = pd.read_sql_query(select_stmt, con=engine)


CPU times: user 690 ms, sys: 69.7 ms, total: 760 ms
Wall time: 20.4 s


In [3]:
source1.head(10)

,ticker,isin,industry,TR_ANALYTICCO2,TR_ANALYTICTOTALWASTE,TR_ENVPRODUCTS,TR_ANALYTICENVRND,TR_ANALYTICWATERUSE,TR_ANALYTICENERGYUSE,TR_POLICYHUMANRIGHTS,TR_POLICYRESPONSIBLEMARKETING,TR_PRODUCTQUALITYMONITORING,TR_POLICYDATAPRIVACY,TR_WOMENEMPLOYEES,TR_AVGTRAININGHOURS,TR_TRADEUNIONREP,TR_ANALYTICLOSTDAYS
0,XAU,NCM0C6I3QYU8,Aerospace and defense,0.88,0.89,0.05,0.15,0.63,0.97,0.45,0.92,0.26,0.82,0.23,0.35,0.01,0.89
1,XBA,CPXE43HILVO2,Professional and commercial services,0.67,0.40,0.33,0.35,0.23,0.55,0.92,NaN,0.04,NaN,0.73,0.72,0.44,0.15
2,XBA,QV9EVZVV7YU1,Investment banking and investment services,0.25,0.51,0.80,0.64,0.82,0.12,0.31,0.17,0.64,0.49,0.71,0.31,0.26,0.84
3,XBAD,FP4MD9IQMNM0,Personal and household,0.28,0.05,NaN,NaN,0.99,0.28,0.41,0.90,0.20,0.63,0.11,0.46,0.80,0.45
4,XBAE,LZAGIYYJVKA3,Industrial conglomerates,0.73,NaN,NaN,0.05,0.36,0.79,0.14,0.37,NaN,NaN,0.66,0.87,0.49,0.38
5,XBF,OYMHZFHXPLV9,Specialty retailers,0.63,0.85,NaN,0.26,0.18,0.39,0.81,0.16,0.29,0.73,0.52,0.95,0.54,0.73
6,XBF,BLCHN91IW6A9,Telecommunications services,0.43,0.97,0.02,0.56,0.33,0.11,0.86,0.76,0.92,0.26,0.03,0.31,0.44,0.62
7,XBH,IE46NA7P03C1,Automobiles and auto parts,0.36,0.88,0.70,0.46,0.23,0.30,0.16,NaN,0.10,0.56,0.93,0.76,0.06,0.91
8,XBHU,RDKLVL3EHRJ1,Freight and logistics services,0.60,0.82,0.29,NaN,NaN,NaN,0.32,0.08,0.64,0.14,0.63,0.15,0.18,0.02
9,XBI,ZET6PCT71084,Transport infrastructure,0.84,0.92,0.89,0.50,0.70,0.96,NaN,NaN,0.53,0.37,0.75,0.34,0.34,0.39


In [4]:
for col in source1.columns:
    new_col = col.replace('TR_', '')
    new_col = new_col.replace('ANALYTIC', '')
    new_col = new_col.capitalize()
    source1.rename({col: new_col}, axis=1, inplace=True)

In [5]:
source1.head(20)

,Ticker,Isin,Industry,Co2,Totalwaste,Envproducts,Envrnd,Wateruse,Energyuse,Policyhumanrights,Policyresponsiblemarketing,Productqualitymonitoring,Policydataprivacy,Womenemployees,Avgtraininghours,Tradeunionrep,Lostdays
0,XAU,NCM0C6I3QYU8,Aerospace and defense,0.88,0.89,0.05,0.15,0.63,0.97,0.45,0.92,0.26,0.82,0.23,0.35,0.01,0.89
1,XBA,CPXE43HILVO2,Professional and commercial services,0.67,0.40,0.33,0.35,0.23,0.55,0.92,NaN,0.04,NaN,0.73,0.72,0.44,0.15
2,XBA,QV9EVZVV7YU1,Investment banking and investment services,0.25,0.51,0.80,0.64,0.82,0.12,0.31,0.17,0.64,0.49,0.71,0.31,0.26,0.84
3,XBAD,FP4MD9IQMNM0,Personal and household,0.28,0.05,NaN,NaN,0.99,0.28,0.41,0.90,0.20,0.63,0.11,0.46,0.80,0.45
4,XBAE,LZAGIYYJVKA3,Industrial conglomerates,0.73,NaN,NaN,0.05,0.36,0.79,0.14,0.37,NaN,NaN,0.66,0.87,0.49,0.38
5,XBF,OYMHZFHXPLV9,Specialty retailers,0.63,0.85,NaN,0.26,0.18,0.39,0.81,0.16,0.29,0.73,0.52,0.95,0.54,0.73
6,XBF,BLCHN91IW6A9,Telecommunications services,0.43,0.97,0.02,0.56,0.33,0.11,0.86,0.76,0.92,0.26,0.03,0.31,0.44,0.62
7,XBH,IE46NA7P03C1,Automobiles and auto parts,0.36,0.88,0.70,0.46,0.23,0.30,0.16,NaN,0.10,0.56,0.93,0.76,0.06,0.91
8,XBHU,RDKLVL3EHRJ1,Freight and logistics services,0.60,0.82,0.29,NaN,NaN,NaN,0.32,0.08,0.64,0.14,0.63,0.15,0.18,0.02
9,XBI,ZET6PCT71084,Transport infrastructure,0.84,0.92,0.89,0.50,0.70,0.96,NaN,NaN,0.53,0.37,0.75,0.34,0.34,0.39


In [6]:
now = datetime.now()
filename = "ESG-consolidated-" + str(now.year) + "-" + str(now.month) + "-" + str(now.day) + "-" + str(now.hour) + ".csv" 
filename

'ESG-consolidated-2024-1-31-9.csv'

In [7]:
COS_ENDPOINT = os.environ.get("COS_ENDPOINT", "ibm-lh-lakehouse-minio-svc.cpd.svc.cluster.local:9000")
COS_ACCESS_KEY = os.environ.get("COS_ACCESS_KEY", "none")
COS_SECRET_KEY = os.environ.get("COS_SECRET_KEY", "none")

minio_client = Minio(
    endpoint=COS_ENDPOINT,
    secure=False,
    access_key=COS_ACCESS_KEY,
    secret_key=COS_SECRET_KEY
)


file_name = filename
bucket = "datamon-bucket"
object_name = "ESG-consolidated.csv"

csv_bytes = source1.to_csv(index=False).encode('utf-8')
csv_buffer = io.BytesIO(csv_bytes)


try:
    
    minio_client.put_object(bucket, object_name, data=csv_buffer,length=len(csv_bytes),content_type='application/csv')
    
except S3Error as e:
    e


In [8]:
"File " + object_name + " uploaded"


'File ESG-consolidated.csv uploaded'